# Trade Report Analysis (V2)

This notebook generates a detailed trade report analysis with de-duplication and performance tracking.

In [10]:
!pip install yfinance pandas tabulate


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [11]:
import sqlite3
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import os
import sys
import contextlib
import io

# --- ROBUST PATH SETUP ---
# Find the project root (where 'app' folder exists)
current_dir = os.getcwd()
project_root = None

# Check current dir and parents
check_path = current_dir
for _ in range(3): # Check up to 3 levels up
    if os.path.exists(os.path.join(check_path, 'app')) and os.path.isdir(os.path.join(check_path, 'app')):
        project_root = check_path
        break
    check_path = os.path.dirname(check_path)

if project_root:
    if project_root not in sys.path:
        sys.path.insert(0, project_root) # Insert at 0 to prioritize
    print(f"Project root found and added: {project_root}")
else:
    print("WARNING: Project root not found. Imports might fail.")
    # Fallback: assume we are in 'notebooks' and root is one up
    fallback = os.path.abspath(os.path.join(current_dir, '..'))
    sys.path.insert(0, fallback)
    print(f"Fallback path added: {fallback}")

# Now import app modules
try:
    from app.services.yahoo_ticker_resolver import YahooTickerResolver
    print("Successfully imported YahooTickerResolver")
except ImportError as e:
    print(f"Import Error: {e}")
    print(f"Current sys.path: {sys.path}")

# DB Path calculation
DB_PATH = os.path.join(project_root if project_root else '..', "subscribers.db")

def get_db_connection():
    return sqlite3.connect(DB_PATH)

def fetch_data():
    conn = get_db_connection()
    query = """
    SELECT 
        id, timestamp, symbol, region, ai_score, recommendation, 
        price_at_decision, deep_research_verdict, batch_id, deep_research_score, batch_winner
    FROM decision_points
    WHERE timestamp >= date('now', '-30 days') -- Optimization: Last 30 days
    ORDER BY timestamp DESC
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

@contextlib.contextmanager
def suppress_output():
    """Context manager to suppress stdout and stderr."""
    save_stdout = sys.stdout
    save_stderr = sys.stderr
    sys.stdout = io.StringIO()
    sys.stderr = io.StringIO()
    try:
        yield
    finally:
        sys.stdout = save_stdout
        sys.stderr = save_stderr

def fetch_yfinance_data(symbols, indices_tickers=['^GSPC', '^DJI', '^GDAXI']):
    resolver = YahooTickerResolver()
    
    # Resolve Symbols
    clean_symbols = []
    symbol_map = {} # Original -> Resolved
    
    print(f"Resolving {len(symbols)} tickers...")
    for s in symbols:
        resolved = resolver.resolve(s)
        clean_symbols.append(resolved)
        symbol_map[s] = resolved
        
    all_tickers = list(set(clean_symbols + indices_tickers))
    
    print(f"Fetching data for {len(all_tickers)} unique tickers...")
    
    with suppress_output():
        data = yf.download(all_tickers, period="1mo", interval="1d", progress=False)['Close']
    
    if isinstance(data, pd.Series):
        data = data.to_frame()
        
    return data, symbol_map

def get_price_on_date(price_data, ticker, date_str):
    try:
        if ticker not in price_data.columns:
            return None
            
        target_date = pd.to_datetime(date_str).tz_localize(None)
        idx = price_data.index.tz_localize(None)
        
        if target_date in idx:
            return price_data.loc[target_date, ticker]
        
        future_dates = idx[idx > target_date]
        if not future_dates.empty:
             return price_data.loc[future_dates[0], ticker]
             
        return None
    except Exception as e:
        return None

def calculate_change(start_price, end_price):
    if start_price is not None and end_price is not None and start_price != 0:
        return (end_price - start_price) / start_price
    return None

def generate_report():
    df = fetch_data()
    
    # 1. Pre-process Dates
    df['date_obj'] = pd.to_datetime(df['timestamp'])
    df['date_str'] = df['date_obj'].dt.strftime('%Y-%m-%d')
    
    # 2. De-duplication
    def rank_row(row):
        score = row['ai_score'] if row['ai_score'] else 0
        has_deep = 1 if row['deep_research_verdict'] and row['deep_research_verdict'] not in ['-', '', 'PENDING', None] else 0
        return (has_deep, score)

    df['rank'] = df.apply(rank_row, axis=1)
    df = df.sort_values(by=['symbol', 'date_str', 'rank'], ascending=[True, True, False])
    
    df_dedup = df.drop_duplicates(subset=['symbol', 'date_str'], keep='first').copy()
    
    print(f"Raw rows: {len(df)}. Deduped rows: {len(df_dedup)}")
    
    # 3. YFinance Data
    symbols = df_dedup['symbol'].unique().tolist()
    price_data, symbol_map = fetch_yfinance_data(symbols)
    
    # 4. Build Report Columns
    report_rows = []
    indices = {'SP500': '^GSPC', 'Dow': '^DJI', 'DAX': '^GDAXI'}
    
    for index, row in df_dedup.iterrows():
        date_str = row['date_str']
        symbol = row['symbol']
        resolved_ticker = symbol_map.get(symbol, symbol)
        
        start_date = row['date_obj']
        end_date = start_date + timedelta(days=7)
        end_date_str = end_date.strftime('%Y-%m-%d')
        
        price_dec = row['price_at_decision']
        price_1w = get_price_on_date(price_data, resolved_ticker, end_date_str)
        
        if datetime.now() < end_date:
            price_1w_disp = "-"
            perf_disp = "-"
            status = "Pending (<1w)"
        else:
            price_1w_disp = f"{price_1w:.2f}" if price_1w is not None else "-"
            change = calculate_change(price_dec, price_1w)
            perf_disp = f"{change*100:+.2f}%" if change is not None else "-"
            status = "Completed" if price_1w is not None else "Data Missing"
            
        bench_res = {}
        for name, ticker in indices.items():
            start_p = get_price_on_date(price_data, ticker, date_str)
            end_p = get_price_on_date(price_data, ticker, end_date_str)
            chg = calculate_change(start_p, end_p)
            bench_res[name] = f"{chg*100:+.2f}%" if chg is not None else "+0.00%"

        verdict = row['deep_research_verdict'] if row['deep_research_verdict'] else "-"
        rec = row['recommendation'] if row['recommendation'] else "-"
        
        if verdict == 'UNKNOWN' or verdict == 'ERROR_PARSING':
             verdict = "ERROR_PARSING"
        
        batch = "Compared" if (row['batch_id'] and row['batch_id'] > 0) else "-"
        if row['batch_winner']:
            batch = "🏆 WINNER"
            
        r = {
            "Date": date_str,
            "Symbol": symbol,
            "Market": row['region'],
            "Score": int(row['ai_score']) if pd.notna(row['ai_score']) else 0,
            "Rec": rec,
            "Price @ Dec": f"{price_dec:.2f}" if price_dec and pd.notna(price_dec) else "0.00",
            "Price +1W": price_1w_disp,
            "Performance": perf_disp,
            "SP500 1W": bench_res['SP500'],
            "Dow 1W": bench_res['Dow'],
            "DAX 1W": bench_res['DAX'],
            "Verdict": verdict,
            "Batch": batch,
            "Status": status
        }
        report_rows.append(r)
        
    report_df = pd.DataFrame(report_rows)
    report_df = report_df.sort_values(by="Date", ascending=False)
    return report_df


Project root found and added: /Users/simonbaumgart/Antigravity/Stock-Tracker
Successfully imported YahooTickerResolver


In [12]:
df = generate_report()
print(df.to_markdown(index=False))
# df.to_csv("trade_report.csv", index=False)

Raw rows: 703. Deduped rows: 673
Resolving 433 tickers...
🔍 Searching Yahoo for: ABEV ()...
🔍 Searching Yahoo for: ABVX ()...
🔍 Searching Yahoo for: ACDSF ()...
🔍 Searching Yahoo for: ADP ()...
🔍 Searching Yahoo for: AEG ()...
🔍 Searching Yahoo for: AFRM ()...
🔍 Searching Yahoo for: AFX ()...
🔍 Searching Yahoo for: AGN ()...
🔍 Searching Yahoo for: AIRA ()...
🔍 Searching Yahoo for: ALAB ()...
🔍 Searching Yahoo for: ALBHF ()...
🔍 Searching Yahoo for: ALBKY ()...
🔍 Searching Yahoo for: ALNY ()...
🔍 Searching Yahoo for: AMG ()...
🔍 Searching Yahoo for: AMS ()...
🔍 Searching Yahoo for: APD ()...
🔍 Searching Yahoo for: APLD ()...
🔍 Searching Yahoo for: APOTEA ()...
🔍 Searching Yahoo for: ARGX ()...
🔍 Searching Yahoo for: ARM ()...
🔍 Searching Yahoo for: ASND ()...
🔍 Searching Yahoo for: ASTS ()...
🔍 Searching Yahoo for: ASX ()...
🔍 Searching Yahoo for: AVAV ()...
🔍 Searching Yahoo for: AVGO ()...
🔍 Searching Yahoo for: AWE ()...
🔍 Searching Yahoo for: AXIA ()...
🔍 Searching Yahoo for: AZN ()